In [2]:
import ast
import sqlite3
import torch
import time
# Connect to the SQLite database
conn = sqlite3.connect('tenhou_dataprocess/dst/2016-2020_after_script_waits.db')
cursor = conn.cursor()
# Query the data
start_time = time.time()
cursor.execute('SELECT X_values, y_values FROM test_table')
rows = cursor.fetchall()

# Process the data
X_extracted = []
y_extracted = []
counter = 0
for row in rows:
    counter += 1
    X_row = ast.literal_eval(row[0])  # Convert string back to list
    y_row = ast.literal_eval(row[1])  # Convert string back to list
    X_extracted.append(X_row)
    y_extracted.append(y_row)
    if (counter //1000 == 700):
        break
    if(counter % 1000 == 0):
        print(f"Processed {counter} rows", end='\r')
    
end = time.time()

# Convert lists back to NumPy arrays or tensors if needed
import numpy as np

X_extracted = np.array(X_extracted)
y_extracted = np.array(y_extracted)

# Optionally, convert to tensors
X = torch.tensor(X_extracted, dtype=torch.float)
y = torch.tensor(y_extracted, dtype=torch.float)

# Close the connection
conn.close()
print(X.shape, y.shape)  # Check shapes of tensors

torch.Size([700000, 368]) torch.Size([700000, 34])


In [4]:

torch.save(X, 'X_extracted_waits_no!.pt')
torch.save(y, 'y_extracted_waits_no!.pt')


In [1]:
import ast
import sqlite3
import torch
import time
X = torch.load('X_extracted_waits.pt')
y = torch.load('y_extracted_waits.pt')


/tmp/ipykernel_130797/1254571462.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load('X_extracted_waits.pt')
/tmp/ipykernel_130797/1254571462.py:6: FutureWarn

In [3]:
print(X[600000])

tensor([  1.,   0.,   0., 370., 175., 259., 196.,  38.,   0.,   2.,   0.,   0.,
          0.,   1.,  32.,  -1.,  -1.,  -1.,  -1.,   1.,   2.,   5.,   5.,   8.,
          8.,  13.,  14.,  19.,  19.,  22.,  23.,  23.,  23.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  31.,  31.,  31.,  -1.,   1.,   1.,   2.,
         -1.,  28.,  28.,  28.,  -1.,   1.,   1.,   3.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,
         -1.,  -1.,  -1.,  -1.,  -1.,  -

In [2]:
#split the data into train, validation and test
index1 = int(len(X)*0.8)
index2 = int(len(X)*0.9)
X_train = X[:index1]
X_val = X[index1:index2]
X_test = X[index2:]
y_train = y[:index1]
y_val = y[index1:index2]
y_test = y[index2:]

In [8]:
import tensorflow as tf

from transformers import  BertModel
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from torch.utils.data import DataLoader
import torch
import random
import time

class MahjongModel(nn.Module):
    def __init__(self):
        super(MahjongModel, self).__init__()
        self.projection = nn.Linear(368, 768)
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc1 = nn.Linear(768, 34)
    def forward(self, x_batch):
        # Convert categorical features to embeddings
        #x_emb = self.embedding(x_batch)  # Shape: (batch_size, seq_length, embedding_dim)
        
        # Project concatenated features to BERT's hidden size
        #x = self.fc1(torch.cat((x_batch, x_emb)))  # Shape: (batch_size, hidden_size)
        #x = self.fc1(x_batch)
        # Add sequence length dimension for BERT
        #x = x.unsqueeze(1)  # Shape: (batch_size, sequence_length=1, hidden_size)
        x_batch = self.projection(x_batch)        
        x_batch = x_batch.unsqueeze(1)
        # Use BERT to process the combined features
        outputs = self.bert(inputs_embeds=x_batch)[0]   
        # Pass through classification head
        outputs = self.fc1(outputs)
        y_hat = torch.sigmoid(outputs)
        return y_hat
    
from torch.utils.data import Dataset 
class MahjongDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
                'input': self.data[idx].clone().detach().float(),
                'label': self.labels[idx].clone().detach().float(),
        }
    
    
def getindices(tensor):
    mylist = []
    for i in range(len(tensor)):
        if tensor[i]:
            mylist.append(i)
    return mylist
    
    

def eval(model, X_val, y_val):
    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        outputs = (outputs >= 0.5).float()
        
        ran = random.randint(222, 1444)
        print(getindices(outputs[ran][0]),getindices(y_val[ran]))
        
        ran = random.randint(2200, 3330)
        print(getindices(outputs[ran][0]),getindices(y_val[ran]))
        
        ran = random.randint(3300, 4000)
        print(getindices(outputs[ran][0]),getindices(y_val[ran]))
        
        
        count = 0
        for i in range(y_val.size(0)):
            count += getindices(outputs[i][0]) == getindices(y_val[i])
        total_samples = y_val.size(0)
        misclassification_rate = 1.0 - (count / total_samples)
        return misclassification_rate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MahjongModel().to(device)

start = time.time()
train_dataset = MahjongDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
print("Time taken to load data is ", time.time()-start)

loss_function = nn.BCELoss().to(device)
optimizer = Adam(model.parameters() ,lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
  


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Time taken to load data is  0.00033783912658691406


In [9]:
for epoch in range(5):
    start = time.time()
    model.train()
    num_batches = 0
    for param in model.bert.parameters():
        param.requires_grad = False
    epoch_loss = 0
    for batch in train_dataloader:
        inputs = batch["input"].to(device)
        labels = batch["label"].to(device)
        outputs = model(inputs)
        outputs = torch.squeeze(outputs).to(device)
        #y_pred_tf = torch.squeeze(outputs).to(device)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        num_batches += 1
        optimizer.step()
        
        epoch_loss += loss.item()  # Accumulate the loss
    print("Epoch ", epoch, " train loss is: ", epoch_loss/num_batches)
    print("Time taken for epoch ", epoch, " is ", time.time()-start)

Epoch  0  train loss is:  0.2369377851704139
Time taken for epoch  0  is  151.2475085258484


KeyboardInterrupt: 

In [15]:
#Real training
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_math_sdp(True)


optimizer = Adam(model.parameters() ,lr=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
  
for epoch in range(100): # can train for as long as you want
    start = time.time()
    model.train()
    for param in model.bert.parameters():
        param.requires_grad = False
    for param in model.bert.encoder.layer[-6:].parameters():
        param.requires_grad = True
    num_batches = 0
    epoch_loss = 0
    
    for batch in train_dataloader:
        inputs = batch["input"].to(device)
        labels = batch["label"].to(device)
        outputs = model(inputs)
        y_pred_tf = torch.squeeze(outputs).to(device)
        loss = loss_function(y_pred_tf, labels)
        optimizer.zero_grad()
        loss.backward()
        num_batches += 1
        optimizer.step()
        scheduler.step(loss)
        epoch_loss += loss.item()
    print("Epoch ", epoch, " train loss is: ", epoch_loss/num_batches, "Time taken for epoch ", epoch, " is ", time.time()-start)
    start = time.time()
    if(epoch % 5 == 2):    
        miss = eval(model, X_val.to(device), y_val.to(device))
        print("Miss is ", miss, "Time taken for eval is ", time.time()-start)  


Epoch  0  train loss is:  0.1967027241558218 Time taken for epoch  0  is  83.06718492507935
Epoch  1  train loss is:  0.19670166857063878 Time taken for epoch  1  is  75.57200646400452
Epoch  2  train loss is:  0.19670130001888414 Time taken for epoch  2  is  71.82989573478699
[] [16]
[] [12, 15]
[] [22, 25]
Miss is  1.0 Time taken for eval is  55.99852967262268
Epoch  3  train loss is:  0.1967018863673184 Time taken for epoch  3  is  71.70641231536865
Epoch  4  train loss is:  0.19670126351512546 Time taken for epoch  4  is  73.72745418548584
Epoch  5  train loss is:  0.19670081146783322 Time taken for epoch  5  is  73.06768012046814
Epoch  6  train loss is:  0.19670089177610234 Time taken for epoch  6  is  73.33236384391785
Epoch  7  train loss is:  0.19670085652545777 Time taken for epoch  7  is  73.12764763832092
[] [18, 31]
[] [23, 28]
[] [13, 16]
Miss is  1.0 Time taken for eval is  60.68917679786682
Epoch  8  train loss is:  0.1966995378135545 Time taken for epoch  8  is  80.555

KeyboardInterrupt: 

In [17]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
print(train_dataset)

In [10]:
model = MahjongModel()
model.load_state_dict(torch.load('model.pth'))
model.to(device)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

MahjongModel(
  (projection): Linear(in_features=368, out_features=768, bias=True)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)

In [6]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
import tensorflow as tf

import numpy as np
from transformers import  BertModel
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from torch.utils.data import DataLoader
import torch
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

clf = TabNetMultiTaskClassifier(cat_idxs=range(34),
                       cat_dims=[1]*34,
                       cat_emb_dim=34,
                       optimizer_fn=Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params=dict(
                            mode='min',
                            factor=0.1,
                            patience=3
                        ),
                       scheduler_fn=ReduceLROnPlateau,
                       mask_type='entmax' # "sparsemax"
                      )


X_train_np = X_train.numpy()
y_train_np = y_train.numpy()
X_val_np = X_val.numpy()
y_val_np = y_val.numpy()
for i in range(20):
    clf.fit(
        X_train=X_train_np, y_train=y_train_np,
        eval_set=[(X_train_np, y_train_np), (X_val_np, y_val_np)],
        eval_name=['train', 'valid'],
        max_epochs=1 , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        drop_last=False,
        loss_fn=[torch.nn.functional.cross_entropy]*34 # Optional, just an example of list usage
    ).to(device)
    print(i)

../aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [39,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [39,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [39,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [39,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [39,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [39,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [39,0,0], thread:

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
